In [1]:
import pandas as pd

def load_ecg_data_and_annotations(ecg_file_path, annotation_file_path):
    ecg_data = pd.read_csv(ecg_file_path)
    annotations = pd.read_csv(annotation_file_path)

    return ecg_data, annotations


/var/folders/x_/27ffh59s2pl353c1p0mccnzc0000gn/T/ipykernel_14290/1582698094.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd
from scipy.signal import butter, filtfilt

def _butter_filter(sequence):
    fs = 360  # Sampling frequency
    nyquist = 0.5 * fs
    low = 0.4 / nyquist
    high = 45 / nyquist

    b, a = butter(N=3, Wn=[low, high], btype='band')
    return filtfilt(b, a, sequence)

def apply_filter(ecg_data):
    filtered_data = ecg_data.copy()
    for lead in ['MLII', 'V1']:
        # Check if the lead is in the DataFrame
        if lead in ecg_data.columns:
            filtered_data[lead] = _butter_filter(ecg_data[lead].values)

    return filtered_data

In [3]:
import numpy as np
from scipy.signal import find_peaks

def detect_r_peaks(ecg_lead, distance=180):
    peaks, _ = find_peaks(ecg_lead, distance=distance)
    return peaks

In [4]:
ecg_data_207, annotations_207 = load_ecg_data_and_annotations('../data/207/207.csv', '../data/207/207annotations.csv')
filtered_ecg_data_207 = apply_filter(ecg_data_207)

In [16]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def prepare_data(filtered_ecg_data_207, annotations_207, window_size=180):
    # Detect R-Peaks
    # window_size = 180
    r_peaks = detect_r_peaks(filtered_ecg_data_207['MLII'].values, distance=window_size)
    
    # Create Segmentations based on R-Peaks
    segment_data = []
    for r_peak in r_peaks:
        start = max(0, r_peak - window_size // 2)
        end = min(len(filtered_ecg_data_207), r_peak + window_size // 2)

        relevant_annotations = annotations_207[(annotations_207['Sample #'] >= start) & (annotations_207['Sample #'] <= end)]
        relevant_annotations = relevant_annotations[relevant_annotations['Type'].isin(['L', 'V', 'A', 'E', '!'])]

        if not relevant_annotations.empty:
            closest_annotation = relevant_annotations.iloc[(relevant_annotations['Sample #'] - r_peak).abs().argsort()[:1]]
            label = closest_annotation['Type'].values[0]
            segment_data.append({'Start': start, 'End': end, 'Label': label})

    segments = pd.DataFrame(segment_data)
    
    # Remove last Row since window size < 180
    segments.drop(segments.tail(1).index,inplace=True)


    # Create arrays from DataFrame
    segments_feature_1 = []
    segments_feature_2 = []
    segment_labels = []
    for index, row in segments.iterrows():
        start_index = int(row['Start'])
        end_index = int(row['End'])
        label = row['Label']
        
        segment_mlII = filtered_ecg_data_207['MLII'][start_index:end_index+1].values
        segment_v1 = filtered_ecg_data_207['V1'][start_index:end_index+1].values
        
        segments_feature_1.append(segment_mlII)
        segments_feature_2.append(segment_v1)
        segment_labels.append(label)

    combined_segments = [np.column_stack((mlII, v1)) for mlII, v1 in zip(segments_feature_1, segments_feature_2)]
    combined_segments_array = np.array([np.array(segment) for segment in combined_segments], dtype=object)

    label_mapping = {'L': 0, 'V': 1, 'A': 2, 'E': 3, '!': 4}
    integer_labels = np.array([label_mapping[label] for label in segment_labels])
    one_hot_labels = to_categorical(integer_labels)

    print(f"Combined Segments Shape: {combined_segments_array.shape}")
    print(f"One-Hot Labels Shape: {one_hot_labels.shape}")

    # Split data into train/test
    train_x, test_x, train_y, test_y = train_test_split(
        combined_segments_array, one_hot_labels, test_size=0.2, random_state=42, stratify=one_hot_labels
    )

    # Display Class Distribution
    integer_labels_from_one_hot = np.argmax(train_y, axis=1)
    class_counts = np.bincount(integer_labels_from_one_hot)
    class_names = ['L', 'V', 'A', 'E', '!']
    for class_name, count in zip(class_names, class_counts):
        print(f"Class {class_name}: {count}")



    # Standardise Train Set
    nsamples, ntimesteps, nfeatures = train_x.shape
    train_x_reshaped = train_x.reshape((nsamples*ntimesteps, nfeatures))
    scaler = StandardScaler()
    scaler.fit(train_x_reshaped)
    train_x_standardised = scaler.transform(train_x_reshaped)
    train_x_standardised = train_x_standardised.reshape((nsamples, ntimesteps, nfeatures))

    return train_x_standardised, train_y, test_x, test_y, scaler



In [7]:
def get_metrics(results, metrics_names, metric_key):
    for name, value in zip(metrics_names, results):
        if metric_key in name:
            return value
    return None

In [8]:
hyperparameter_space = {
    'window_size': [180, 280, 380],
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'lstm_units': [32, 64],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.1, 0.01, 0.001],
    'num_lstm_layers': [2, 3, 4],
    'reg_learning_rate': [0.1, 0.01, 0.001]
}

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_and_train_model(train_x, train_y, dropout_rate, lstm_units, batch_size, learning_rate, reg_learning_rate, num_lstm_layers, window_size, val_x=[], val_y=[]):
    model = Sequential()
    model.add(LSTM(lstm_units, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2]),
                   kernel_regularizer=l2(reg_learning_rate), 
                   recurrent_regularizer=l2(reg_learning_rate)))
    
    for i in range(1, num_lstm_layers):
        model.add(LSTM(lstm_units, return_sequences=True if i < num_lstm_layers - 1 else False,
                       kernel_regularizer=l2(reg_learning_rate), 
                       recurrent_regularizer=l2(reg_learning_rate)))
        
    model.add(Dropout(dropout_rate))
    model.add(Dense(train_y.shape[1], activation='softmax', kernel_regularizer=l2(reg_learning_rate)))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])
    model.fit(train_x, train_y, validation_data=(val_x, val_y) if len(val_x) != 0 else None, epochs=30, batch_size=batch_size, verbose=1)

    if len(val_x) != 0:
        results = model.evaluate(val_x, val_y, verbose=0)
    else:
        results = model.evaluate(train_x, train_y, verbose=0)
    metrics_names = model.metrics_names

    accuracy = results[metrics_names.index('accuracy')]
    precision = get_metrics(results, metrics_names, 'precision')
    recall = get_metrics(results, metrics_names, 'recall')
    
    return model, accuracy, precision, recall


In [17]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

n_iterations = 2
best_score = 0
best_params = {}
best_test_x = []
best_test_y = []
best_scaler = None
best_train_x = []
best_train_y = []

for iteration in range(n_iterations):
    chosen_params = { param: np.random.choice(values) for param, values in hyperparameter_space.items() }
    print(f"Current Hyperparameters: {chosen_params}")

    train_x_standardised, train_y, test_x, test_y, scaler = prepare_data(filtered_ecg_data_207, annotations_207, window_size=chosen_params['window_size'])

    y_labels = np.argmax(train_y, axis=1)

    accuracies = []
    precisions = []
    recalls = []
    for train_index, val_index in kf.split(train_x_standardised, y_labels):
        train_x_fold, val_x_fold = train_x_standardised[train_index], train_x_standardised[val_index]
        train_y_fold, val_y_fold = train_y[train_index], train_y[val_index]

        model, accuracy, precision, recall = build_and_train_model(train_x_fold, train_y_fold, **chosen_params, val_x=val_x_fold, val_y=val_y_fold,)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)

        print(f"Fold Scores: Acc - {accuracy} Pr - {precision} Re - {recall}")
    
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    print(f"Current Mean Scores: Acc - {avg_accuracy} Pr - {avg_precision} Re - {avg_recall} , Current Hyperparameters: {chosen_params}")

    # Update best params etc.
    if avg_accuracy > best_score:
        best_score = avg_accuracy
        best_params = chosen_params
        best_metrics = {
            'accuracy': avg_accuracy,
            'precision': avg_precision,
            'recall': avg_recall
        }
        best_test_x = test_x
        best_test_y = test_y
        best_scaler = scaler
        best_train_x = train_x_standardised
        best_train_y = train_y
        print(f"New best score: {avg_accuracy:.4f} with params: {best_params} and metrics: {best_metrics}")

# Final best results
print(f"Best score: {best_score:.4f}")
print(f"Best params: {best_params}")
print(f"Best metrics: {best_metrics}")

# TODO:
# Try out balancing (weighted, or oversampling)
# Try out diff. window sizes


Current Hyperparameters: {'window_size': 280, 'dropout_rate': 0.5, 'lstm_units': 64, 'batch_size': 32, 'learning_rate': 0.001, 'num_lstm_layers': 2, 'reg_learning_rate': 0.001}
Combined Segments Shape: (1710, 281, 2)
One-Hot Labels Shape: (1710, 5)
Class L: 1078
Class V: 59
Class A: 38
Class E: 84
Class !: 109
Epoch 1/30
13/35 [==========>...................] - ETA: 3s - loss: 1.4501 - accuracy: 0.7788 - precision_1: 0.8214 - recall_1: 0.1659    

KeyboardInterrupt: 

In [41]:
print(f"Window Size: {best_params['window_size']}")
print(f"Dropout Rate: {best_params['dropout_rate']}")
print(f"LSTM Layers: {best_params['num_lstm_layers']}")
print(f"LSTM Units: {best_params['lstm_units']}")
print(f"Batch Size: {best_params['batch_size']}")
print(f"Learning Rate: {best_params['learning_rate']}")
print(f"Regularizer Learning Rate: {best_params['reg_learning_rate']}")

# Train final model with the best parameters
model, accuracy, precision, recall = build_and_train_model(train_x_standardised, train_y, **best_params)

Dropout Rate: 0.2
LSTM Layers: 4
LSTM Units: 32
Batch Size: 64
Learning Rate: 0.001
Regularizer Learning Rate: 0.001


In [42]:
# Standardise Test Set
test_nsamples, test_ntimesteps, test_nfeatures = test_x.shape
test_x_reshaped = test_x.reshape((test_nsamples * test_ntimesteps, test_nfeatures))
test_x_standardised = scaler.transform(test_x_reshaped)
test_x_standardised = test_x_standardised.reshape((test_nsamples, test_ntimesteps, test_nfeatures))

In [43]:
# Evaluate on Test Set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_x_standardised, test_y, verbose=1)
test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1_score}")

14/14 [==============================] - 1s 46ms/step - loss: 0.3578 - accuracy: 0.9220 - precision_29: 0.9407 - recall_29: 0.9007
Test Loss: 0.35776078701019287
Test Accuracy: 0.9219858050346375
Test Precision: 0.9407407641410828
Test Recall: 0.9007092118263245
Test F1 Score: 0.9202898620770421


In [44]:
from sklearn.metrics import classification_report

predictions = model.predict(test_x_standardised)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_y, axis=1)
report = classification_report(true_classes, predicted_classes, target_names=['L', 'V', 'A', 'E', '!'])
print(report)

14/14 [==============================] - 3s 46ms/step
              precision    recall  f1-score   support

           L       0.96      0.98      0.97       313
           V       0.69      0.48      0.56        23
           A       0.54      0.47      0.50        15
           E       0.93      0.87      0.90        30
           !       0.86      0.90      0.88        42

    accuracy                           0.92       423
   macro avg       0.79      0.74      0.76       423
weighted avg       0.92      0.92      0.92       423

